In [ ]:
include(joinpath(dirname(Base.find_package("Globtim")), "..", ".globtim", "notebook_setup.jl"))

We set the number of samples used to generate the approximant. It is annoying that the error goes up while the degree has increased.

In [2]:
# Constants and Parameters
const n, a, b = 3, 1, 1
const scale_factor = a / b
f = Rastringin  
rand_center = [0.0, 0.0, 0.0];
d = 10 # initial degree 
SMPL = 40 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor 
                )

test_input(3, [0.0, 0.0, 0.0], 40, (0.1, 0.5), 0.002, (0.0, 0.0), 1.0, 1.0, 6, Globtim.Rastringin)

Solve the system of partial derivatives using `Homotopy_COntinuation.jl`. 

In [3]:
pol_cheb = Constructor(TR, d, basis=:chebyshev, precision=RationalPrecision)
@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 0.06249859921441929


In [4]:
@polyvar(x[1:n]) # Define polynomial ring 
real_pts_cheb = solve_polynomial_system(
    x, n, d, pol_cheb.coeffs;
    basis=pol_cheb.basis,
    precision=pol_cheb.precision,
    normalized=pol_cheb.normalized,
)

Tracking 729 paths...   0%|▏                            |  ETA: 0:26:50



Tracking 729 paths... 100%|█████████████████████████████| Time: 0:00:04
                   # paths tracked: 729
   # non-singular solutions (real): 729 (125)
       # singular endpoints (real): 0 (0)
          # total solutions (real): 729 (125)


125-element Vector{Vector{Float64}}:
 [0.9911840615254303, 0.9911840615254277, 0.9911840615254267]
 [-1.2954874092360923e-17, 0.9911840615254319, 0.9911840615254268]
 [-0.9911840615254334, 0.9911840615254331, 0.9911840615254264]
 [-0.5024836441161775, 0.9911840615254326, 0.9911840615254269]
 [0.5024836441161776, 0.9911840615254351, 0.9911840615254273]
 [0.99118406152543, -3.321282484512236e-16, 0.9911840615254317]
 [-1.0170985797104464e-17, -3.411578738452561e-16, 0.9911840615254288]
 [-0.9911840615254319, -3.2409520648058375e-16, 0.9911840615254293]
 [-0.5024836441161775, -3.3501623567716955e-16, 0.9911840615254286]
 [0.5024836441161775, -3.3610781504996206e-16, 0.9911840615254275]
 ⋮
 [-6.6116728720583654e-18, -0.502483644116178, 0.5024836441161774]
 [-0.9911840615254285, -0.5024836441161783, 0.5024836441161774]
 [-0.5024836441161774, -0.5024836441161781, 0.5024836441161773]
 [0.5024836441161775, -0.502483644116178, 0.5024836441161772]
 [0.9911840615254288, 0.5024836441161773, 0.5024

In [5]:
df_cheb = process_crit_pts(real_pts_cheb, f, TR)

Row,x1,x2,x3,z
,Float64,Float64,Float64,Float64
1,0.991184,0.991184,0.991184,-27.0066
2,-1.29549e-17,0.991184,0.991184,-28.0044
3,-0.991184,0.991184,0.991184,-27.0066
4,-0.502484,0.991184,0.991184,-7.75316
5,0.502484,0.991184,0.991184,-7.75316
6,0.991184,-3.32128e-16,0.991184,-28.0044
7,-1.0171e-17,-3.41158e-16,0.991184,-29.0022
8,-0.991184,-3.24095e-16,0.991184,-28.0044
9,-0.502484,-3.35016e-16,0.991184,-8.75094


In [6]:
using Optim
# Enhanced analysis with new statistics
tol_dist = 0.05
df_cheb_enhanced, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=tol_dist, verbose=true)

println("\n=== Enhanced Statistics Summary ===")
println("Enhanced DataFrame columns: $(names(df_cheb_enhanced))")
println("Enhanced DataFrame size: $(size(df_cheb_enhanced))")

if nrow(df_min_cheb) > 0
    println("Minimizers DataFrame columns: $(names(df_min_cheb))")
    println("Minimizers DataFrame size: $(size(df_min_cheb))")
    
    println("\n=== Enhanced Statistics Column Documentation ===")
    println("Critical Points DataFrame Columns:")
    println("  x1, x2, x3        - Original critical point coordinates")
    println("  z                 - Function value at critical point")
    println("  y1, y2, y3        - BFGS optimized coordinates from critical point")
    println("  close             - Whether optimized point is close to starting point (tol_dist=$tol_dist)")
    println("  steps             - Number of BFGS optimization iterations")
    println("  converged         - Whether BFGS converged within domain bounds")
    println("  region_id         - Spatial region ID (domain divided into cubic regions)")
    println("  function_value_cluster - Cluster ID based on function value similarity")
    println("  nearest_neighbor_dist  - Distance to nearest other critical point")
    println("  gradient_norm     - ||∇f(x)|| at critical point (should be ~0)")
    
    println("\nUnique Minimizers DataFrame Columns:")
    println("  x1, x2, x3              - Coordinates of unique minimizer")
    println("  value                   - Function value at minimizer")
    println("  captured                - Whether minimizer was captured by a critical point")
    println("  basin_points            - Number of critical points converging to this minimizer")
    println("  average_convergence_steps - Average BFGS steps for points reaching this minimizer")
    println("  region_coverage_count   - Number of different spatial regions feeding this minimizer")
    println("  gradient_norm_at_min    - ||∇f(x)|| at minimizer (should be very close to 0)")

    println("\n=== Sample Enhanced Data ===")
    println("First 5 rows of enhanced critical points:")
    println(first(df_cheb_enhanced, 5))
    
    println("\nUnique minimizers with enhanced data:")
    println(df_min_cheb)
    
else
    println("No unique minimizers found")
end

Processing point 1 of 125
Optimization has converged within bounds: ✓
Processing point 2 of 125
Optimization has converged within bounds: ✓
Processing point 3 of 125
Optimization has converged within bounds: ✓
Processing point 4 of 125
Optimization has converged within bounds: ✓
Processing point 5 of 125
Optimization has converged within bounds: ✓
Processing point 6 of 125
Optimization has converged within bounds: ✓
Processing point 7 of 125
Optimization has converged within bounds: ✓
Processing point 8 of 125
Optimization has converged within bounds: ✓
Processing point 9 of 125
Optimization has converged within bounds: ✓
Processing point 10 of 125
Optimization has converged within bounds: ✓
Processing point 11 of 125
Optimization has converged within bounds: ✓
Processing point 12 of 125
Optimization has converged within bounds: ✓
Processing point 13 of 125
Optimization has converged within bounds: ✓
Processing point 14 of 125
Optimization has converged within bounds: ✓
Processing poin

In [ ]:
using GLMakie
GLMakie.activate!()  # Ensure GLMakie is active for 3D plots
include("../../src/LevelSetViz.jl")

In [8]:
grid = scale_factor * generate_grid(3, 120)  # 3D grid
values = map(f, grid); # Prepare level set data for specific level

In [9]:
fig = create_level_set_visualization(f, grid, df_cheb_enhanced, (-30.0, 30.))

In [10]:
display(fig)

GLMakie.Screen(...)

In [ ]:
# GLMakie.closeall()